In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime as dt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly import tools
import missingno as msno
import warnings
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon,MultiPolygon

from pyproj import Proj, transform
import pandas as pd

warnings.filterwarnings('ignore')
mpl.rc('font', family = 'Malgun Gothic')
sns.set(style='darkgrid')

C:\Users\dblab\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dblab\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\dblab\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 20년 주택매매데이터 불러오기 및 사용할 컬럼 지정

In [2]:
#1,2,3,4분기
def return_quarter(month):
    if(month <=3):
        return 1
    elif((month >3) & (month<=6)):
        return 2
    elif((month >6) & (month<=9)):
        return 3
    elif((month >9) & (month<=12)):
        return 4

In [3]:
주택매매데이터 = pd.read_csv('../point_Polygon_전처리후_데이터/2020년주택매매데이터.csv').drop('Unnamed: 0',axis=1)
주택매매데이터 = 주택매매데이터[['계약_년','계약_월','거래금액(만원)','전용면적(㎡)','층','건축년도','행정구','행정동']]
주택매매데이터 = 주택매매데이터.rename(columns = {'계약_년':'년도','계약_월':'월'})
주택매매데이터['거래금액(만원)'] = 주택매매데이터['거래금액(만원)'].apply(lambda x: str(x.replace(',','')))
주택매매데이터['거래금액(만원)'] = 주택매매데이터['거래금액(만원)'].astype('int')
주택매매데이터['분기'] = 주택매매데이터['월'].apply(return_quarter)
주택매매데이터

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2
1,2020,2,144000,54.98,4,1987,강남구,개포4동,1
2,2020,6,162750,79.97,4,1987,강남구,개포4동,2
3,2020,6,160000,79.97,5,1987,강남구,개포4동,2
4,2020,6,150000,79.97,2,1987,강남구,개포4동,2
...,...,...,...,...,...,...,...,...,...
83329,2020,10,43000,59.96,1,1999,중랑구,신내1동,4
83330,2020,11,46000,59.96,11,1999,중랑구,신내1동,4
83331,2020,12,65750,84.87,14,1999,중랑구,신내1동,4
83332,2020,9,40000,77.71,5,2007,중랑구,중화1동,3


In [4]:
#기본 정보 확인
주택매매데이터.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83334 entries, 0 to 83333
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   년도        83334 non-null  int64  
 1   월         83334 non-null  int64  
 2   거래금액(만원)  83334 non-null  int32  
 3   전용면적(㎡)   83334 non-null  float64
 4   층         83334 non-null  int64  
 5   건축년도      83334 non-null  int64  
 6   행정구       83334 non-null  object 
 7   행정동       83334 non-null  object 
 8   분기        83334 non-null  int64  
dtypes: float64(1), int32(1), int64(5), object(2)
memory usage: 5.4+ MB


# 테이블 merge

## 구별 학생수

In [5]:
구별학생수 = pd.read_csv('../통계데이터_전처리후/구별학급당학생수.csv').drop('Unnamed: 0',axis=1)
data = pd.merge(주택매매데이터,구별학생수,how='left',on=['년도','행정구'])
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,유치원학급수,초등학생수,초등학교학급수,중학생수,중학교학급수,고등학생수,고등학교학급수
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,181,25905,990,15602,555,17837,717
1,2020,2,144000,54.98,4,1987,강남구,개포4동,1,3260,181,25905,990,15602,555,17837,717
2,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,181,25905,990,15602,555,17837,717
3,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,181,25905,990,15602,555,17837,717
4,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,181,25905,990,15602,555,17837,717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83329,2020,10,43000,59.96,1,1999,중랑구,신내1동,4,2700,154,14554,696,6649,300,6150,273
83330,2020,11,46000,59.96,11,1999,중랑구,신내1동,4,2700,154,14554,696,6649,300,6150,273
83331,2020,12,65750,84.87,14,1999,중랑구,신내1동,4,2700,154,14554,696,6649,300,6150,273
83332,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,154,14554,696,6649,300,6150,273


In [6]:
data.isnull().sum().sum()

0

## 구별 의료기관 merge

In [7]:
구별의료기관 = pd.read_csv('../통계데이터_전처리후/구별의료기관.csv').drop('Unnamed: 0',axis=1)
data = pd.merge(data,구별의료기관,how='left',on=['년도','행정구'])
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,고등학생수,고등학교학급수,총병원수,총병상수,총종합병원수,총종합병원병상수,총일반병원수,총일반병원병상수,총의원수,총의원병상수
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,17837,717,2693,8637,3,3013,32,2040,1697,1388
1,2020,2,144000,54.98,4,1987,강남구,개포4동,1,3260,...,17837,717,2693,8637,3,3013,32,2040,1697,1388
2,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,17837,717,2693,8637,3,3013,32,2040,1697,1388
3,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,17837,717,2693,8637,3,3013,32,2040,1697,1388
4,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,17837,717,2693,8637,3,3013,32,2040,1697,1388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83329,2020,10,43000,59.96,1,1999,중랑구,신내1동,4,2700,...,6150,273,531,3238,3,1045,13,1015,244,579
83330,2020,11,46000,59.96,11,1999,중랑구,신내1동,4,2700,...,6150,273,531,3238,3,1045,13,1015,244,579
83331,2020,12,65750,84.87,14,1999,중랑구,신내1동,4,2700,...,6150,273,531,3238,3,1045,13,1015,244,579
83332,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,6150,273,531,3238,3,1045,13,1015,244,579


In [8]:
data.isnull().sum().sum()

0

## 동별주차장 merge

In [9]:
동별주차장 = pd.read_csv('../통계데이터_전처리후/동별주차장.csv').drop('Unnamed: 0',axis=1)
동별주차장 = 동별주차장.drop('자치구',axis=1)
data = pd.merge(data,동별주차장,how='left',on = ['년도','행정동'])
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,총병원수,총병상수,총종합병원수,총종합병원병상수,총일반병원수,총일반병원병상수,총의원수,총의원병상수,주차장_개소,주차장_면수
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,2693,8637,3,3013,32,2040,1697,1388,987,9172
1,2020,2,144000,54.98,4,1987,강남구,개포4동,1,3260,...,2693,8637,3,3013,32,2040,1697,1388,987,9172
2,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,2693,8637,3,3013,32,2040,1697,1388,987,9172
3,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,2693,8637,3,3013,32,2040,1697,1388,987,9172
4,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,2693,8637,3,3013,32,2040,1697,1388,987,9172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83532,2020,10,43000,59.96,1,1999,중랑구,신내1동,4,2700,...,531,3238,3,1045,13,1015,244,579,495,15800
83533,2020,11,46000,59.96,11,1999,중랑구,신내1동,4,2700,...,531,3238,3,1045,13,1015,244,579,495,15800
83534,2020,12,65750,84.87,14,1999,중랑구,신내1동,4,2700,...,531,3238,3,1045,13,1015,244,579,495,15800
83535,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,531,3238,3,1045,13,1015,244,579,668,4742


In [10]:
set(주택매매데이터['행정동']) - set(동별주차장['행정동']) 

set()

In [11]:
data.isnull().sum().sum()

0

## 동별 지하철역수 merge

In [12]:
동별지하철역수 = pd.read_csv('../통계데이터_전처리후/동별지하철역수.csv')
data = pd.merge(data,동별지하철역수,how='left',on = '행정동')
data['동별 지하철역 수'] = data['동별 지하철역 수'].fillna(0)
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,총병상수,총종합병원수,총종합병원병상수,총일반병원수,총일반병원병상수,총의원수,총의원병상수,주차장_개소,주차장_면수,동별 지하철역 수
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,8637,3,3013,32,2040,1697,1388,987,9172,0.0
1,2020,2,144000,54.98,4,1987,강남구,개포4동,1,3260,...,8637,3,3013,32,2040,1697,1388,987,9172,0.0
2,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,8637,3,3013,32,2040,1697,1388,987,9172,0.0
3,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,8637,3,3013,32,2040,1697,1388,987,9172,0.0
4,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,8637,3,3013,32,2040,1697,1388,987,9172,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83532,2020,10,43000,59.96,1,1999,중랑구,신내1동,4,2700,...,3238,3,1045,13,1015,244,579,495,15800,1.0
83533,2020,11,46000,59.96,11,1999,중랑구,신내1동,4,2700,...,3238,3,1045,13,1015,244,579,495,15800,1.0
83534,2020,12,65750,84.87,14,1999,중랑구,신내1동,4,2700,...,3238,3,1045,13,1015,244,579,495,15800,1.0
83535,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,3238,3,1045,13,1015,244,579,668,4742,0.0


In [13]:
data.isnull().sum().sum()

0

## 동별 평균 연령

In [14]:
동별평균연령 = pd.read_csv('../통계데이터_전처리후/동별평균연령.csv').drop('Unnamed: 0',axis=1)
data = pd.merge(data,동별평균연령,how='inner',left_on = ['행정구','행정동','년도','분기'],right_on = ['자치구','행정동','년도','분기'])
data = data.drop('자치구',axis=1)
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,총일반병원수,총일반병원병상수,총의원수,총의원병상수,주차장_개소,주차장_면수,동별 지하철역 수,전체평균연령,남자_평균연령,여자_평균연령
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,32,2040,1697,1388,987,9172,0.0,42.2,41.3,43.0
1,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,32,2040,1697,1388,987,9172,0.0,42.2,41.3,43.0
2,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,32,2040,1697,1388,987,9172,0.0,42.2,41.3,43.0
3,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,32,2040,1697,1388,987,9172,0.0,42.2,41.3,43.0
4,2020,6,165000,67.28,2,1987,강남구,개포4동,2,3260,...,32,2040,1697,1388,987,9172,0.0,42.2,41.3,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83454,2020,8,60000,59.76,18,1998,중랑구,중화1동,3,2700,...,13,1015,244,579,668,4742,0.0,44.1,43.2,45.0
83455,2020,7,44000,95.94,6,2007,중랑구,중화1동,3,2700,...,13,1015,244,579,668,4742,0.0,44.1,43.2,45.0
83456,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,13,1015,244,579,668,4742,0.0,44.1,43.2,45.0
83457,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,13,1015,244,579,668,4742,0.0,44.1,43.2,45.0


In [15]:
print(set(data['행정동']) - set(동별평균연령['행정동']))
print(set(동별평균연령['행정동']) - set(data['행정동']))
print(data.isnull().sum().sum())

set()
{'삼청동', '둔촌1동', '가회동'}
0


## 동별 세대원수별 세대수

In [16]:
동별세대원수별세대수 = pd.read_csv('../통계데이터_전처리후/서울 세대원수별 세대수.csv').drop('Unnamed: 0',axis=1)
data = pd.merge(data,동별세대원수별세대수,how='inner',left_on = ['행정구','행정동','년도','분기'],right_on = ['자치구','행정동','년도','분기'])
data = data.drop('자치구',axis=1)
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,전체평균연령,남자_평균연령,여자_평균연령,전체세대수,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,42.2,41.3,43.0,7768,2978,1598,1466,1323,312,63
1,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,42.2,41.3,43.0,7768,2978,1598,1466,1323,312,63
2,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,42.2,41.3,43.0,7768,2978,1598,1466,1323,312,63
3,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,42.2,41.3,43.0,7768,2978,1598,1466,1323,312,63
4,2020,6,165000,67.28,2,1987,강남구,개포4동,2,3260,...,42.2,41.3,43.0,7768,2978,1598,1466,1323,312,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83454,2020,8,60000,59.76,18,1998,중랑구,중화1동,3,2700,...,44.1,43.2,45.0,9092,3421,2274,1736,1290,296,46
83455,2020,7,44000,95.94,6,2007,중랑구,중화1동,3,2700,...,44.1,43.2,45.0,9092,3421,2274,1736,1290,296,46
83456,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,44.1,43.2,45.0,9092,3421,2274,1736,1290,296,46
83457,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,44.1,43.2,45.0,9092,3421,2274,1736,1290,296,46


In [17]:
print(set(data['행정동']) - set(동별세대원수별세대수['행정동']))
print(set(동별세대원수별세대수['행정동']) - set(data['행정동']))
print(data.isnull().sum().sum())

set()
{'삼청동', '둔촌1동', '가회동'}
0


## 동별 외국인 데이터

In [18]:
동별외국인데이터 = pd.read_csv('../통계데이터_전처리후/서울 외국인 데이터(20년도1분기~20년도4분기).csv').drop('Unnamed: 0',axis=1)
data = pd.merge(data,동별외국인데이터,how='inner',left_on = ['행정구','행정동','년도','분기'],right_on = ['자치구','행정동','년도','분기'])
data = data.drop('자치구',axis=1)
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,남자_평균연령,여자_평균연령,전체세대수,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,총 외국인 수
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,41.3,43.0,7768,2978,1598,1466,1323,312,63,166
1,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,41.3,43.0,7768,2978,1598,1466,1323,312,63,166
2,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,41.3,43.0,7768,2978,1598,1466,1323,312,63,166
3,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,41.3,43.0,7768,2978,1598,1466,1323,312,63,166
4,2020,6,165000,67.28,2,1987,강남구,개포4동,2,3260,...,41.3,43.0,7768,2978,1598,1466,1323,312,63,166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83454,2020,8,60000,59.76,18,1998,중랑구,중화1동,3,2700,...,43.2,45.0,9092,3421,2274,1736,1290,296,46,180
83455,2020,7,44000,95.94,6,2007,중랑구,중화1동,3,2700,...,43.2,45.0,9092,3421,2274,1736,1290,296,46,180
83456,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,43.2,45.0,9092,3421,2274,1736,1290,296,46,180
83457,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,43.2,45.0,9092,3421,2274,1736,1290,296,46,180


In [19]:
print(set(data['행정동']) - set(동별세대원수별세대수['행정동']))
print(set(동별세대원수별세대수['행정동']) - set(data['행정동']))
print(data.isnull().sum().sum())

set()
{'삼청동', '둔촌1동', '가회동'}
0


## 동별 대학교 개수

In [20]:
동별대학교개수 = pd.read_csv('../통계데이터_전처리후/서울시 대학 및 전문대학 DB 정보 (한국어).csv')
data = pd.merge(data,동별대학교개수,how='left',left_on = ['행정구','행정동'],right_on = ['자치구','행정동'])
data = data.drop('자치구',axis=1)
data = data.rename(columns = {'키값':'대학수'})
data['대학수'] = data['대학수'].fillna(0)
data


,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,여자_평균연령,전체세대수,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,총 외국인 수,대학수
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,43.0,7768,2978,1598,1466,1323,312,63,166,0.0
1,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,43.0,7768,2978,1598,1466,1323,312,63,166,0.0
2,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,43.0,7768,2978,1598,1466,1323,312,63,166,0.0
3,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,43.0,7768,2978,1598,1466,1323,312,63,166,0.0
4,2020,6,165000,67.28,2,1987,강남구,개포4동,2,3260,...,43.0,7768,2978,1598,1466,1323,312,63,166,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83454,2020,8,60000,59.76,18,1998,중랑구,중화1동,3,2700,...,45.0,9092,3421,2274,1736,1290,296,46,180,0.0
83455,2020,7,44000,95.94,6,2007,중랑구,중화1동,3,2700,...,45.0,9092,3421,2274,1736,1290,296,46,180,0.0
83456,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,45.0,9092,3421,2274,1736,1290,296,46,180,0.0
83457,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,45.0,9092,3421,2274,1736,1290,296,46,180,0.0


In [21]:
# print(set(data['행정동']) - set(동별대학교개수['행정동']))
print(set(동별대학교개수['행정동']) - set(data['행정동']))
print(data.isnull().sum().sum())

set()
0


## 동별 기초수급자

In [22]:
동별기초수급자 = pd.read_csv('../통계데이터_전처리후/서울시 동별 기초생활 수급자.csv').drop('Unnamed: 0',axis=1)
동별기초수급자['총 수급자 수'] = 동별기초수급자['총 수급자 수'].apply(lambda x: str(x.replace(',','')))
동별기초수급자['총 수급자 수'] = 동별기초수급자['총 수급자 수'].astype('int')
data = pd.merge(data,동별기초수급자,how='left',left_on = ['년도','행정구','행정동'],right_on = ['년도','자치구','행정동'])
data = data.drop('자치구',axis=1)
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,전체세대수,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,총 외국인 수,대학수,총 수급자 수
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,7768,2978,1598,1466,1323,312,63,166,0.0,441
1,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,7768,2978,1598,1466,1323,312,63,166,0.0,441
2,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,7768,2978,1598,1466,1323,312,63,166,0.0,441
3,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,7768,2978,1598,1466,1323,312,63,166,0.0,441
4,2020,6,165000,67.28,2,1987,강남구,개포4동,2,3260,...,7768,2978,1598,1466,1323,312,63,166,0.0,441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83454,2020,8,60000,59.76,18,1998,중랑구,중화1동,3,2700,...,9092,3421,2274,1736,1290,296,46,180,0.0,682
83455,2020,7,44000,95.94,6,2007,중랑구,중화1동,3,2700,...,9092,3421,2274,1736,1290,296,46,180,0.0,682
83456,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,9092,3421,2274,1736,1290,296,46,180,0.0,682
83457,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,9092,3421,2274,1736,1290,296,46,180,0.0,682


In [23]:
print(set(data['행정동']) - set(동별기초수급자['행정동']))
print(set(동별기초수급자['행정동']) - set(data['행정동']))
print(data.isnull().sum().sum())

set()
{'삼청동', '가회동'}
0


## 동별 노령화

In [24]:
동별노령화 = pd.read_csv('../통계데이터_전처리후/서울시 동별 노령화 데이터.csv').drop('Unnamed: 0',axis=1)
data = pd.merge(data,동별노령화,how='left',left_on = ['년도','행정구','행정동'],right_on = ['년도','자치구','행정동'])
data = data.drop('자치구',axis=1)
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,3인세대,4인세대,5인세대,6인세대,총 외국인 수,대학수,총 수급자 수,유년부양비,노년부양비,노령화지수
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,1466,1323,312,63,166,0.0,441,15.6,17.4,111.5
1,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,1466,1323,312,63,166,0.0,441,15.6,17.4,111.5
2,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,1466,1323,312,63,166,0.0,441,15.6,17.4,111.5
3,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,1466,1323,312,63,166,0.0,441,15.6,17.4,111.5
4,2020,6,165000,67.28,2,1987,강남구,개포4동,2,3260,...,1466,1323,312,63,166,0.0,441,15.6,17.4,111.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83454,2020,8,60000,59.76,18,1998,중랑구,중화1동,3,2700,...,1736,1290,296,46,180,0.0,682,13.0,23.4,180.1
83455,2020,7,44000,95.94,6,2007,중랑구,중화1동,3,2700,...,1736,1290,296,46,180,0.0,682,13.0,23.4,180.1
83456,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,1736,1290,296,46,180,0.0,682,13.0,23.4,180.1
83457,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,1736,1290,296,46,180,0.0,682,13.0,23.4,180.1


In [25]:
print(set(data['행정동']) - set(동별기초수급자['행정동']))
print(set(동별기초수급자['행정동']) - set(data['행정동']))
print(data.isnull().sum().sum())

set()
{'삼청동', '가회동'}
0


## 동별 인구밀도

In [26]:
동별인구밀도 = pd.read_csv('../통계데이터_전처리후/서울시 인구밀도(동별)통계.csv').drop('Unnamed: 0',axis=1)
data = pd.merge(data,동별인구밀도,how='left',left_on = ['년도','행정구','행정동'],right_on = ['년도','자치구','행정동'])
data = data.drop('자치구',axis=1)
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,6인세대,총 외국인 수,대학수,총 수급자 수,유년부양비,노년부양비,노령화지수,인구 수,면적,인구밀도
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,63,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279
1,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,63,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279
2,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,63,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279
3,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,63,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279
4,2020,6,165000,67.28,2,1987,강남구,개포4동,2,3260,...,63,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83454,2020,8,60000,59.76,18,1998,중랑구,중화1동,3,2700,...,46,180,0.0,682,13.0,23.4,180.1,20400,0.64,31875
83455,2020,7,44000,95.94,6,2007,중랑구,중화1동,3,2700,...,46,180,0.0,682,13.0,23.4,180.1,20400,0.64,31875
83456,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,46,180,0.0,682,13.0,23.4,180.1,20400,0.64,31875
83457,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,46,180,0.0,682,13.0,23.4,180.1,20400,0.64,31875


In [27]:
print(set(data['행정동']) - set(동별기초수급자['행정동']))
print(set(동별기초수급자['행정동']) - set(data['행정동']))
print(data.isnull().sum().sum())

set()
{'삼청동', '가회동'}
0


## 동별주택종류별통계

In [28]:
동별주택종류별통계 = pd.read_csv('../통계데이터_전처리후/주택종류별(동별)통계.csv').drop('Unnamed: 0',axis=1)
data = pd.merge(data,동별주택종류별통계,how='left',left_on = ['년도','행정구','행정동'],right_on = ['년도','자치구','행정동'])
data = data.drop('자치구',axis=1)
data

,년도,월,거래금액(만원),전용면적(㎡),층,건축년도,행정구,행정동,분기,유치원생_수,...,총 외국인 수,대학수,총 수급자 수,유년부양비,노년부양비,노령화지수,인구 수,면적,인구밀도,전체종류주택합계
0,2020,6,149500,77.75,4,1988,강남구,개포4동,2,3260,...,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279,7409
1,2020,6,162750,79.97,4,1987,강남구,개포4동,2,3260,...,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279,7409
2,2020,6,160000,79.97,5,1987,강남구,개포4동,2,3260,...,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279,7409
3,2020,6,150000,79.97,2,1987,강남구,개포4동,2,3260,...,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279,7409
4,2020,6,165000,67.28,2,1987,강남구,개포4동,2,3260,...,166,0.0,441,15.6,17.4,111.5,22765,1.49,15279,7409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83454,2020,8,60000,59.76,18,1998,중랑구,중화1동,3,2700,...,180,0.0,682,13.0,23.4,180.1,20400,0.64,31875,5691
83455,2020,7,44000,95.94,6,2007,중랑구,중화1동,3,2700,...,180,0.0,682,13.0,23.4,180.1,20400,0.64,31875,5691
83456,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,180,0.0,682,13.0,23.4,180.1,20400,0.64,31875,5691
83457,2020,9,40000,77.71,5,2007,중랑구,중화1동,3,2700,...,180,0.0,682,13.0,23.4,180.1,20400,0.64,31875,5691


In [29]:
print(set(data['행정동']) - set(동별기초수급자['행정동']))
print(set(동별기초수급자['행정동']) - set(data['행정동']))
print(data.isnull().sum().sum())

set()
{'삼청동', '가회동'}
0


# 테이블 저장

In [30]:
data.to_csv('../main_데이터_코드/main_data.csv',encoding='utf-8-sig')